In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import gcsfs
from google.cloud import bigquery, storage
from google.oauth2 import credentials

In [2]:
def get_blob(blobs):
    for blob in blobs:
        yield blob

In [ ]:
image_dir='/spectrain/images/'
# plt.figure(figsize=(300,40))

# initialize the GCS client
storage_client = storage.Client()

# get the storage bucket
bucket = storage_client.get_bucket('spectrain')

# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs('spectrain', prefix='Kidney_TX_Data')

i = 0 # counter to use for breaking

# Note: The call returns a response only when the iterator is consumed.
for blob in get_blob(blobs):
    if("output" in blob.name):
        plt.figure(figsize=(300,40), clear=True)
        df=pd.read_csv("gs://spectrain/"+blob.name, sep='\s', header=None)
        df.columns=['x_axis', 'y_axis']
        plt.plot(df['x_axis'], df['y_axis'])
        plt.axis('off')
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        filename = blob.name.split('.')[0] # remove the suffix/file extension
        filename = filename.split('/')[1] # remove the containing directory name from filename
        upload_blob = bucket.blob(image_dir+filename+'_nmr.png')
        upload_blob.upload_from_file(buf, content_type='image/png', rewind=True)
        buf.close()
        i=i+1
        if i==30:
            break